In [137]:
import torch
from torch import nn
import torchvision
import pandas as pd
import numpy as np

In [138]:
# device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### 1. Data Processing

In [139]:
train_df = pd.read_csv("mnist_train_final.csv")
test_df = pd.read_csv("mnist_test_final.csv")

In [140]:
y_train = train_df["label"].values
y_encoded = []
for y in y_train:
    zeros = np.zeros(10)
    zeros[y] = 1
    y_encoded.append(zeros)

y_train = np.array(y_encoded)

y_test = test_df["label"].values
y_encoded = []
for y in y_test:
    zeros = np.zeros(10)
    zeros[y] = 1
    y_encoded.append(zeros)

y_test = np.array(y_encoded)


In [141]:
X_train = torch.from_numpy(train_df.drop(["label"], axis=1).values).type(torch.float32).to(device)
y_train = torch.from_numpy(y_train).type(torch.float32).to(device)
X_test = torch.from_numpy(test_df.drop(["label"], axis=1).values).type(torch.float32).to(device)
y_test = torch.from_numpy(y_test).type(torch.float32).to(device)

In [142]:
X_train = X_train.reshape((59999, 1, 28, 28))
X_test = X_test.reshape((9999, 1, 28, 28))

### 2. Create Model

In [143]:
# define model
class DigitClassifier(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer_stack = nn.Sequential(
            nn.Conv2d(1, 28, 3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2)),

            nn.Conv2d(28, 64, 3),
            nn.ReLU(),

            nn.Conv2d(64, 16, 3),
            nn.ReLU(),

            nn.Flatten(),

            nn.Linear(81, 128),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(128, 10)
        )
    
    def forward(self, x):
        return self.layer_stack(x)
    
model = DigitClassifier().to(device)

In [145]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)

In [146]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

### 3. Train Model

In [150]:
# set seed
torch.manual_seed(42)

epochs = 10

for epoch in range(epochs):

    # TRAINING
    model.train()

    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    acc = accuracy_fn(y_train, y_pred)


    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    

    # TESTING
    model.eval()

    with torch.inference_mode():
        test_pred = model(X_test)
        test_loss = loss_fn(y_pred, test_pred)
        test_acc = accuracy_fn(test_pred, y_pred)

    
    print(f"Epoch {epoch} | Train Loss: {loss:.4f} | Train Accuracy: {acc:.2f}% | Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.2f}%")

OutOfMemoryError: CUDA out of memory. Tried to allocate 4.23 GiB. GPU 0 has a total capacity of 6.00 GiB of which 0 bytes is free. Of the allocated memory 9.59 GiB is allocated by PyTorch, and 1.34 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)